In [ ]:
# *******************************************************************************************************************************************
# 2. The Blastp analysis : preparation step 
# *******************************************************************************************************************************************

    # The makeblastdb command : 
makeblastdb -in /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB.fasta \
-dbtype prot  \
-title depolymerase_db \
-out /home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB


    # The command for the multifasta faa for the congruent and the loners :
import os
import subprocess
from multiprocessing.pool import ThreadPool

path_70="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/blast_out_phageboost_70_80"

path_fasta = [f"{path_70}/{strain}/tmp/{phage}/{fasta}" 
              for strain in os.listdir(f"{path_70}") 
              for phage in os.listdir(f"{path_70}/{strain}/tmp") 
              for fasta in os.listdir(f"{path_70}/{strain}/tmp/{phage}")]

def blast_bea(path) :
    #path_out = "/".join(path.split("/")[0:-1])
    file_name = path.split("/")[-1].split(".fasta")[0]
    blastp = f"blastp -query {path}  -db {path_db} -out {path_out}/{file_name}.out -outfmt 6"
    sub_blastp = subprocess.Popen(blastp, shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    press_out, press_err = sub_blastp.communicate()
    print(press_out)
    
    
def show_result(protein) :
    outfmt = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart" ,"send", "evalue", "bitscore"]
    strain = protein.split("__")[0]
    prophage = protein.split("__")[1]
    results = pd.read_csv(f"{path_out}/{protein}.out", names = outfmt, sep = "\t")
    return results

    
if __name__ == '__main__':
    with ThreadPool(10) as p:
        p.map(blast_bea, path_fasta)
        
        
        

In [ ]:
import os
import subprocess
from multiprocessing.pool import ThreadPool
import pandas as pd
from tqdm import tqdm

path_70="/home/conchae/prediction_depolymerase_tropism/prophage_prediction/prophage_decipher/phageboost/phageboost_70_20102022"
path_db = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db/bead_depo/bea_depo.DB"
path_out = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/blast_out_phageboost_70_80"
path_write = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher/bea_all_depolymerase.db"
path_work = "/home/conchae/prediction_depolymerase_tropism/prophage_prediction/depolymerase_decipher"


outfmt = ["qseqid", "sseqid", "pident", "length", "mismatch", "gapopen", "qstart", "qend", "sstart" ,"send", "evalue", "bitscore"]
depo_prot = set()
def scan_blast(path_results) :
    with open(f"{path_work}/full_prediction.vsBeaDB.txt", "a+") as outfile :
        if os.path.getsize(path_results) >0 :
            results = pd.read_csv(f"{path_results}", names = outfmt, sep = "\t")
            if results.iloc[0]["bitscore"] > 75 :
                depo_prot.add(results.iloc[0]["qseqid"])
                outfile.write(f"{results.iloc[0]['qseqid']}\t{results.iloc[0]['bitscore']}\n")
            
            
paths = [f"{path_out}/{file}" for file in os.listdir(path_out)]        
        
if __name__ == '__main__':
    with ThreadPool(30) as p:
        p.map(scan_blast, paths)
        